# Basic Chat Conversation - No Memory

In [4]:
#!pip install langgraph langchain[openai]

In [1]:
from google.colab import drive
import os
drive.mount('/content/drive', force_remount = True)

api_key_file = '/content/drive/MyDrive/api_key.txt'
with open(api_key_file, 'r', encoding = 'utf-8-sig') as file:
    api_key = file.read().strip()

print(api_key is not None)
os.environ['OPENAI_API_KEY'] = api_key
print("OpenAI API key loaded:", os.environ.get('OPENAI_API_KEY') is not None)

Mounted at /content/drive
True
OpenAI API key loaded: True


In [8]:
from langgraph.graph import StateGraph ,START, END
from typing_extensions import TypedDict, List
from langchain_openai import ChatOpenAI
from langchain_core.messages import HumanMessage, AIMessage, BaseMessage
import os

llm = ChatOpenAI(model = 'gpt-4o-mini', temperature = 0)

class StateChat(TypedDict):
	messages: List[BaseMessage]           # Bases Messages -> Human, AI, System

def answer_node(state: StateChat)-> dict:
    '''
    Take human message & return AI response.
    '''
    human:HumanMessage = state['messages'][-1]
    ai_response:AIMessage = llm.invoke(input = [human])
    return {'messages': [ai_response]}


# Building Graph (Node + Edges)
builder = StateGraph(StateChat)
builder.add_node('_answer_', answer_node)
builder.add_edge(start_key = START, end_key = '_answer_')
builder.add_edge(start_key = '_answer_', end_key = END)
graph = builder.compile()


def response(message:str, history:List[List[str]])-> str:  # [[user_message, ai_response], [user_message, ai_response], [user_message, ai_response]...]
    '''
    For Gradio usage.
    '''
    response = graph.invoke({'messages': [HumanMessage(content = message)]})
    response_text = response['messages'][0].content
    history = history + [[message, response_text]]                                   # adding to the actual history
    return "", history

In [10]:
# Test
graph.invoke({'messages': [HumanMessage(content = 'What is the height of Mount Everest?')]})['messages'][0].content

'Mount Everest has a height of 8,848.86 meters (29,031.7 feet) above sea level, as officially recognized by China and Nepal in December 2020.'